<a href="https://colab.research.google.com/github/chetools/CHE4071_Fall2025/blob/main/dynamic_distillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/chetools/chetools/main/tools/che5.ipynb -O che5.ipynb
!pip install importnb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.7 MB/s eta 0:00:00


In [2]:
from importnb import Notebook
with Notebook():
    from che5 import sim, pid, TF1, TF2, shift

import numpy as np
import scipy as sp
import scipy.signal as sig
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "plotly_dark"

import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)

import sympy
from sympy.abc import s
from sympy import exp, Symbol, simplify


In [3]:
#[0C, 1, 2, 3, 4B]

In [45]:
N=13
Nfeed = 7
alpha = 2
F = 1.  #mol/s feed flow rate
z = 0.5 #mol fraction of more volatile component in feed
q = 0.9  #liquid mole fraction of feed
Dsp = 0.5 * F  #fraction of feed desired as distillate flow
R = 2. #reflux ratio

Vstrip = Dsp + R*Dsp -  F*(1-q)  # vapor flow in stripping section
Vrec = Vstrip +   F*(1-q)  #vapor in rectifying section
m0tray = 0.2 #tray holdup in moles at zero flow
m0cond = 0.5 #condenser holdup at no flow
m0boil = 0.5 #boiler holdup at no flow
m0 = np.r_[m0cond, (m0tray,)*N, m0boil]
m_ic = np.full(N+2,10.)  #fill up condenser, reboiler, and trays with 0.8 moles of liquid
x_ic = np.full(N+2,z) #fill up all stages with feed composition
V = np.r_[0., (Vrec,)*Nfeed, (Vstrip,)*(N-Nfeed+1)]
weir_const = 0.1 #lumped Weir constant that includes effect of weir width, density, viscoisty, conversion from volumetric to molar flows

In [37]:
def rhs_m_only(t, vec):
    m = vec
    mdiff = m-m0
    L=weir_const*(np.where(mdiff<0, 0., mdiff))**(1.5)
    D = L[0]/(R+1)
    L[0] = R*D
    dm=np.zeros(N+2)
    dm-=L
    dm[0]-=D
    dm[1:]+=L[:-1]
    dm[Nfeed]+=F
    dm[:-1]+=V[1:]
    dm[1:]-=V[1:]
    return dm

In [46]:
tend=100
res=sp.integrate.solve_ivp(rhs_m_only, (0,tend), m_ic, method='Radau', dense_output=True)
tplot = np.linspace(0,tend,200)
m_sims = res.sol(tplot)
fig = make_subplots()
for i,m_sim in enumerate(m_sims):
    fig.add_scatter(x=tplot, y=m_sim, name=f'{i}')
fig.update_layout(width=800,height=500)

In [48]:
#alpha = K1/K2 = y1*x2 / (x1*y2) = y1*(1-x1) / (x1*(1-y1))
#alpha*x  =   y- y*x + alpha*x*y = y*(1 - x + alpha*x)
# y = alpha*x/ (1 - x + alpha*x)

def rhs(t, vec):
    x, m = np.split(vec,2)
    y = alpha*x/ (1 - x + alpha*x)
    mdiff = m-m0
    L=weir_const*(np.where(mdiff<0, 0., mdiff))**(1.5)
    D = L[0]/(R+1)
    L[0] = R*D
    dm=np.zeros(N+2)
    dm-=L
    dm[0]-=D
    dm[1:]+=L[:-1]
    dm[Nfeed]+=F
    dm[:-1]+=V[1:]
    dm[1:]-=V[1:]

    #m is number of moles, x is mole fraction of more volatile component
    #m*x is number of moles of more volatile component at each stage (condenser, tray, or reboiler)
    dmx=np.zeros(N+2)
    dmx-=L*x
    dmx[0]-=D*x[0]
    dmx[1:]+=L[:-1]*x[:-1]
    dmx[Nfeed]+=F*z
    dmx[:-1]+= V[1:] * y[1:]
    dmx[1:]-= V[1:] * y[1:]

    dx = (dmx - x*dm)/m
    return np.r_[dx, dm]

In [49]:
tend=100
res=sp.integrate.solve_ivp(rhs, (0,tend), np.r_[x_ic, m_ic], method='Radau', dense_output=True)

In [50]:
tplot = np.linspace(0,tend,200)
x_sims, m_sims = np.split(res.sol(tplot),2,axis=0)
fig = make_subplots()
for i,m_sim in enumerate(m_sims):
    fig.add_scatter(x=tplot, y=m_sim, name=f'{i}')
fig.update_layout(width=800,height=500)

  message: The solver successfully reached the end of the integration interval.
  success: True
   status: 0
        t: [ 0.000e+00  1.219e-01 ...  9.288e+01  1.000e+02]
        y: [[ 5.000e-01  5.030e-01 ...  8.988e-01  9.024e-01]
            [ 5.000e-01  5.000e-01 ...  8.201e-01  8.255e-01]
            ...
            [ 1.000e+01  1.000e+01 ...  7.321e+00  7.320e+00]
            [ 1.000e+01  9.851e+00 ...  3.425e+00  3.424e+00]]
      sol: <scipy.integrate._ivp.common.OdeSolution object at 0x7af5550d6d50>
 t_events: None
 y_events: None
     nfev: 109
     njev: 3
      nlu: 24